## Setting Up Conda Environment for GEE

Follow these steps to create a dedicated conda environment for Google Earth Engine work:

In [ ]:
# Step 1: Create a new conda environment
# Run this command in your terminal (not in this notebook):
# conda create -n gee-env python=3.9 -y

# Step 2: Activate the environment
# conda activate gee-env

# Step 3: Install required packages
# conda install -c conda-forge earthengine-api jupyter notebook ipykernel -y

# Step 4: Install additional useful packages for geospatial work
# conda install -c conda-forge geopandas folium matplotlib seaborn pandas numpy -y

# Step 5: Add the environment to Jupyter
# python -m ipykernel install --user --name gee-env --display-name "Python (GEE)"

print("Copy and run these commands in your terminal to set up the environment!")
print("After setup, restart VS Code and select the 'Python (GEE)' kernel for this notebook.")

#### Import Libraries and Initialize GEE


In [2]:
import ee
import folium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image, display
import json

# Set up matplotlib for inline plotting
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_style("whitegrid")

print("Libraries imported successfully!")
print("Next step: Authenticate and initialize Google Earth Engine")

Libraries imported successfully!
Next step: Authenticate and initialize Google Earth Engine


#### Authenticate Google Earth Engine


In [6]:
# Authenticate Earth Engine - this will open a browser window for login
try:
    ee.Authenticate()
    print("Authentication successful!")
except Exception as e:
    print(f"Authentication failed: {e}")
    print("Make sure you have a Google Earth Engine account and internet connection.")


Successfully saved authorization token.
Authentication successful!


In [2]:
# Initialize Earth Engine
try:
    ee.Initialize()
    print("Google Earth Engine initialized successfully!")
    print(f"EE version: {ee.__version__}")
except Exception as e:
    print(f"Initialization failed: {e}")
    print("Run the authentication cell above first.")

Google Earth Engine initialized successfully!
EE version: 1.6.3


In [ ]:
'''
When opening a new notebook, paste this code for streamlined setup
'''
import sys
sys.path.append('./Setup')
from gee_setup import *

setup_gee()
# Now you're ready to use GEE!

## Helper Functions for GEE Visualization



In [9]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Add a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add the Earth Engine layer method to folium map
folium.Map.add_ee_layer = add_ee_layer

def create_map(center=[0, 0], zoom=2):
    """Create a folium map with Earth Engine layer support."""
    m = folium.Map(location=center, zoom_start=zoom, control_scale=True)
    return m

def get_image_collection_info(collection_name, start_date='2020-01-01', end_date='2023-12-31'):
    """Get basic information about an Earth Engine ImageCollection."""
    try:
        collection = ee.ImageCollection(collection_name).filterDate(start_date, end_date)
        size = collection.size().getInfo()
        first_image = ee.Image(collection.first())
        bands = first_image.bandNames().getInfo()
        
        print(f"Collection: {collection_name}")
        print(f"Date range: {start_date} to {end_date}")
        print(f"Number of images: {size}")
        print(f"Bands available: {bands}")
        
        return collection
    except Exception as e:
        print(f"Error accessing collection {collection_name}: {e}")
        return None

print("Helper functions created successfully!")

Helper functions created successfully!


## Example 1: Basic Satellite Imagery

Let's start with a simple example - loading and visualizing Landsat 8 satellite imagery:

In [3]:
# Define area of interest (AOI) - San Francisco Bay Area
aoi = ee.Geometry.Rectangle([-122.5, 37.3, -121.8, 38.0])

# Load Landsat 8 Collection 2 Surface Reflectance
landsat = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate('2023-06-01', '2023-08-31') \
    .filterBounds(aoi) \
    .filter(ee.Filter.lt('CLOUD_COVER', 10)) \
    .median()

# Define visualization parameters for true color
vis_params_true = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],  # Red, Green, Blue
    'min': 7000,
    'max': 12000,
    'gamma': 1.4
}

# Define visualization parameters for false color (vegetation)
vis_params_false = {
    'bands': ['SR_B5', 'SR_B4', 'SR_B3'],  # NIR, Red, Green
    'min': 7000,
    'max': 12000,
    'gamma': 1.4
}

# Create map centered on San Francisco
map_center = [37.65, -122.15]
m = create_map(center=map_center, zoom=10)

# Add layers to map
m.add_ee_layer(landsat, vis_params_true, 'Landsat 8 - True Color')
m.add_ee_layer(landsat, vis_params_false, 'Landsat 8 - False Color')

# Add layer control
folium.LayerControl().add_to(m)

print("Map created with Landsat 8 imagery!")
print("True color shows natural colors, False color highlights vegetation in red.")
m

EEException: Earth Engine client library not initialized. See http://goo.gle/ee-auth.

## Example 2: Calculate NDVI (Vegetation Index)

Let's calculate and visualize the Normalized Difference Vegetation Index:

In [ ]:
# Calculate NDVI using Landsat 8 bands
# NDVI = (NIR - Red) / (NIR + Red)
ndvi = landsat.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')

# Define NDVI visualization parameters
vis_params_ndvi = {
    'min': -1,
    'max': 1,
    'palette': ['red', 'yellow', 'green']
}

# Create a new map for NDVI
m_ndvi = create_map(center=map_center, zoom=10)

# Add NDVI layer
m_ndvi.add_ee_layer(ndvi, vis_params_ndvi, 'NDVI')

# Add a colorbar legend (simulated)
print("NDVI Map created!")
print("Red areas: Low/no vegetation (water, urban)")
print("Yellow areas: Moderate vegetation")
print("Green areas: High vegetation density")

m_ndvi

## Example 3: Time Series Analysis

Let's create a time series chart of NDVI over time for a specific location:

In [ ]:
# Define a point of interest (Golden Gate Park)
point = ee.Geometry.Point([-122.4694, 37.7694])

# Create a function to add NDVI band to each image
def add_ndvi(image):
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    return image.addBands(ndvi)

# Load Landsat 8 collection for time series (2022-2023)
landsat_ts = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate('2022-01-01', '2023-12-31') \
    .filterBounds(point) \
    .filter(ee.Filter.lt('CLOUD_COVER', 20)) \
    .map(add_ndvi)

# Extract NDVI time series at the point
def extract_ndvi(image):
    ndvi_value = image.select('NDVI').reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=30
    ).get('NDVI')
    
    return ee.Feature(None, {
        'date': image.date().format('YYYY-MM-dd'),
        'ndvi': ndvi_value
    })

# Get the time series data
ndvi_ts = landsat_ts.map(extract_ndvi)
ndvi_list = ndvi_ts.getInfo()

# Convert to pandas DataFrame for plotting
dates = []
ndvi_values = []

for feature in ndvi_list['features']:
    if feature['properties']['ndvi'] is not None:
        dates.append(feature['properties']['date'])
        ndvi_values.append(feature['properties']['ndvi'])

df = pd.DataFrame({
    'date': pd.to_datetime(dates),
    'ndvi': ndvi_values
})

# Plot time series
plt.figure(figsize=(12, 6))
plt.plot(df['date'], df['ndvi'], 'o-', linewidth=2, markersize=6)
plt.title('NDVI Time Series - Golden Gate Park, San Francisco', fontsize=14, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('NDVI', fontsize=12)
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print(f"Time series extracted for {len(df)} dates")
print(f"NDVI range: {df['ndvi'].min():.3f} to {df['ndvi'].max():.3f}")
print(f"Mean NDVI: {df['ndvi'].mean():.3f}")

## Example 4: Land Cover Classification

Let's explore global land cover data from the ESA WorldCover dataset:

In [ ]:
# Load ESA WorldCover 2021 land cover map
worldcover = ee.ImageCollection('ESA/WorldCover/v200').first()

# Define land cover visualization parameters
landcover_vis = {
    'bands': ['Map'],
    'min': 10,
    'max': 100,
    'palette': [
        '#006400',  # Tree cover
        '#ffbb22',  # Shrubland
        '#ffff4c',  # Grassland
        '#f096ff',  # Cropland
        '#fa0000',  # Built-up
        '#b4b4b4',  # Bare/sparse vegetation
        '#f0f0f0',  # Snow and ice
        '#0064c8',  # Permanent water bodies
        '#0096a0',  # Herbaceous wetland
        '#00cf75',  # Mangroves
        '#fae6a0'   # Moss and lichen
    ]
}

# Create map for land cover
m_lc = create_map(center=map_center, zoom=8)
m_lc.add_ee_layer(worldcover, landcover_vis, 'ESA WorldCover 2021')

# Add legend information
legend_html = '''
<div style="position: fixed; 
            bottom: 50px; left: 50px; width: 200px; height: 280px; 
            background-color: white; border:2px solid grey; z-index:9999; 
            font-size:14px; padding: 10px">
<p><b>Land Cover Legend</b></p>
<p><i class="fa fa-square" style="color:#006400"></i> Tree cover</p>
<p><i class="fa fa-square" style="color:#ffbb22"></i> Shrubland</p>
<p><i class="fa fa-square" style="color:#ffff4c"></i> Grassland</p>
<p><i class="fa fa-square" style="color:#f096ff"></i> Cropland</p>
<p><i class="fa fa-square" style="color:#fa0000"></i> Built-up</p>
<p><i class="fa fa-square" style="color:#b4b4b4"></i> Bare/sparse</p>
<p><i class="fa fa-square" style="color:#0064c8"></i> Water bodies</p>
<p><i class="fa fa-square" style="color:#0096a0"></i> Wetland</p>
<p><i class="fa fa-square" style="color:#00cf75"></i> Mangroves</p>
</div>
'''

m_lc.get_root().html.add_child(folium.Element(legend_html))

print("Land cover map created!")
print("This shows global land cover classification for 2021")
m_lc

## Next Steps and Additional Resources

Now you have a working GEE environment! Here are some ideas for further exploration:

### Popular GEE Datasets to Explore:
- **Sentinel-2**: High-resolution optical imagery (`COPERNICUS/S2_SR`)
- **MODIS**: Moderate resolution, frequent coverage (`MODIS/006/MOD13Q1`)
- **Climate Data**: Temperature, precipitation (`ECMWF/ERA5_LAND/HOURLY`)
- **Digital Elevation**: SRTM DEM (`USGS/SRTMGL1_003`)
- **Forest Loss**: Global Forest Change (`UMD/hansen/global_forest_change_2023_v1_11`)

### Useful GEE Functions to Learn:
- `ee.Reducer` functions for statistical analysis
- `ee.Filter` for data filtering
- `ee.Join` for combining datasets
- Machine learning classifiers (`ee.Classifier`)
- Export functions (`Export.image`, `Export.table`)

### Resources:
- [GEE Documentation](https://developers.google.com/earth-engine)
- [GEE Code Editor](https://code.earthengine.google.com/)
- [GEE Community Tutorials](https://developers.google.com/earth-engine/tutorials/community/intro)

Happy Earth Engine coding! 🌍